In [1]:
import tensorflow as tf
import csv
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras

In [2]:
def read_points(dataset_name, label_col_number):
    points = []
    labels = []
    with open (dataset_name, 'r') as fin:
        csvReader = csv.reader(fin, delimiter=',')
        for row in csvReader:
            if csvReader.line_num != 1:
                float_row = []
                for i in range(0, len(row)):
                    if i != label_col_number:
                        float_row.append(float(row[i]) * 1.0 / 255)
                points.append(np.reshape(float_row, [28, 28]))
                labels.append(int(row[label_col_number]))
    return points, labels

In [3]:
points, labels = read_points("../Datasets/mnist.csv", 0)

In [4]:
def model(features, labels, mode, params):
    if params["function"]:
        input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
        conv1 = keras.layers.Conv2D(8, [3, 3], padding="same", activation=params["function"])(input_layer)
        conv2 = keras.layers.Conv2D(8, [3, 3], padding="same", activation=params["function"])(conv1)
        conv3 = keras.layers.Conv2D(8, [3, 3], padding="same", activation=params["function"])(conv2)
    
        conv3_reshaped = tf.reshape(conv3, [-1, 28 * 28 * 8])
    
        fc1 = keras.layers.Dense(64, activation=params["function"])(conv3_reshaped)
        fc2 = keras.layers.Dense(64, activation=params["function"])(fc1)
        logits = keras.layers.Dense(10)(fc2)
    else:
        input_layer = tf.reshape(features["x"], [-1, 28 * 28 * 1])
    
        fc1 = keras.layers.Dense(64, activation=tf.nn.sigmoid)(input_layer)
        fc2 = keras.layers.Dense(10)(fc1)

        logits = fc2
    
    predictions = {
        "classes": tf.argmax(input=logits, axis=1)
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(loss=loss,global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])
    }
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [5]:
def train_and_validate(train_points, train_labels, test_points, test_labels, dir_name, N, function=None):
    mnist_classifier = tf.estimator.Estimator(
        model_fn=model, model_dir=dir_name, params={"function" : function})
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": train_points},
        y=train_labels,
        batch_size=100,
        num_epochs=None,
        shuffle=True)
    for epoch in range(N):
        mnist_classifier.train(input_fn=train_input_fn, steps=100)
        eval_input_fn_train = tf.estimator.inputs.numpy_input_fn(
            x={"x": train_points},
            y=train_labels,
            num_epochs=1,
            shuffle=False)
        eval_results_train = mnist_classifier.evaluate(input_fn=eval_input_fn_train)
        
        eval_input_fn_test = tf.estimator.inputs.numpy_input_fn(
            x={"x": test_points},
            y=test_labels,
            num_epochs=1,
            shuffle=False)
        eval_results_test = mnist_classifier.evaluate(input_fn=eval_input_fn_test)
        print("Epoch " + str(epoch) + ": train accuracy = " + str(eval_results_train["accuracy"]) + ", test accuracy = " + str(eval_results_test["accuracy"]))

In [6]:
import random
def split(points, labels):
    test_indexes = random.sample(range(len(points)), len(points) // 10)
    train_indexes = set([i for i in range(len(points))]) - set(test_indexes)
    return np.array([points[i] for i in train_indexes]), np.array([labels[i] for i in train_indexes]), np.array([points[i] for i in test_indexes]), np.array([labels[i] for i in test_indexes])

In [7]:
train_points, train_labels, test_points, test_labels = split(points, labels)

In [8]:
tf.logging.set_verbosity(tf.logging.ERROR)

In [9]:
train_and_validate(train_points, train_labels, test_points, test_labels, "mnist_model_sigmoid", 20)

Epoch 0: train accuracy = 0.29066667, test accuracy = 0.289
Epoch 1: train accuracy = 0.5283333, test accuracy = 0.523
Epoch 2: train accuracy = 0.61766666, test accuracy = 0.615
Epoch 3: train accuracy = 0.66344446, test accuracy = 0.668
Epoch 4: train accuracy = 0.6911111, test accuracy = 0.702
Epoch 5: train accuracy = 0.69755554, test accuracy = 0.712
Epoch 6: train accuracy = 0.7313333, test accuracy = 0.744
Epoch 7: train accuracy = 0.74355555, test accuracy = 0.761
Epoch 8: train accuracy = 0.7548889, test accuracy = 0.771
Epoch 9: train accuracy = 0.7678889, test accuracy = 0.786
Epoch 10: train accuracy = 0.779, test accuracy = 0.8
Epoch 11: train accuracy = 0.7874445, test accuracy = 0.812
Epoch 12: train accuracy = 0.794, test accuracy = 0.818
Epoch 13: train accuracy = 0.8021111, test accuracy = 0.825
Epoch 14: train accuracy = 0.80644447, test accuracy = 0.829
Epoch 15: train accuracy = 0.81144446, test accuracy = 0.833
Epoch 16: train accuracy = 0.81766665, test accuracy 

In [10]:
train_and_validate(train_points, train_labels, test_points, test_labels, "mnist_model_tanh", 20, tf.tanh)

Epoch 0: train accuracy = 0.8252222, test accuracy = 0.845
Epoch 1: train accuracy = 0.8778889, test accuracy = 0.883
Epoch 2: train accuracy = 0.8976667, test accuracy = 0.897
Epoch 3: train accuracy = 0.90544444, test accuracy = 0.903
Epoch 4: train accuracy = 0.9122222, test accuracy = 0.913
Epoch 5: train accuracy = 0.9167778, test accuracy = 0.91
Epoch 6: train accuracy = 0.9221111, test accuracy = 0.919
Epoch 7: train accuracy = 0.9301111, test accuracy = 0.926
Epoch 8: train accuracy = 0.934, test accuracy = 0.927
Epoch 9: train accuracy = 0.9368889, test accuracy = 0.93
Epoch 10: train accuracy = 0.9391111, test accuracy = 0.929
Epoch 11: train accuracy = 0.94255555, test accuracy = 0.932
Epoch 12: train accuracy = 0.9448889, test accuracy = 0.933
Epoch 13: train accuracy = 0.94755554, test accuracy = 0.933
Epoch 14: train accuracy = 0.95066667, test accuracy = 0.938
Epoch 15: train accuracy = 0.95555556, test accuracy = 0.937
Epoch 16: train accuracy = 0.955, test accuracy = 0

In [11]:
train_and_validate(train_points, train_labels, test_points, test_labels, "mnist_model_relu", 20, tf.nn.relu)

Epoch 0: train accuracy = 0.44788888, test accuracy = 0.442
Epoch 1: train accuracy = 0.8951111, test accuracy = 0.904
Epoch 2: train accuracy = 0.90533334, test accuracy = 0.916
Epoch 3: train accuracy = 0.91533333, test accuracy = 0.919
Epoch 4: train accuracy = 0.928, test accuracy = 0.937
Epoch 5: train accuracy = 0.93733335, test accuracy = 0.935
Epoch 6: train accuracy = 0.9411111, test accuracy = 0.936
Epoch 7: train accuracy = 0.9494445, test accuracy = 0.952
Epoch 8: train accuracy = 0.95244443, test accuracy = 0.949
Epoch 9: train accuracy = 0.95044446, test accuracy = 0.952
Epoch 10: train accuracy = 0.9584444, test accuracy = 0.961
Epoch 11: train accuracy = 0.9577778, test accuracy = 0.954
Epoch 12: train accuracy = 0.96511114, test accuracy = 0.954
Epoch 13: train accuracy = 0.96955556, test accuracy = 0.959
Epoch 14: train accuracy = 0.97077775, test accuracy = 0.961
Epoch 15: train accuracy = 0.9682222, test accuracy = 0.956
Epoch 16: train accuracy = 0.96566665, test a